# 作业02：Fashion Mnist实现图像分类

## 1 概述
Fashion-MNIST是一个替代MNIST手写数字集的图像数据集，它是由Zalando（一家德国的时尚科技公司）旗下的研究部门提供。其涵盖了来自10种类别的共7万个不同商品的正面图片。Fashion-MNIST的大小、格式和训练集/测试集划分与原始的MNIST完全一致。数据集按照60000/10000的比例进行训练测试数据划分，全部数据均为28x28的灰度图片。

**基本要求**：
1. 使用卷积神经网络完成Fashion-MNIST的分类任务。
2. 在README文件中描述所使用的模型的结构、优化器、损失函数和超参数等信息，以及模型在训练集和测试集上的最优结果。
3. 对模型中训练过程损失函数的变化趋势可视化

**加分项**：可以对模型进行优化（包括增加层数，使用残差连接，调整超参数等），或者对模型的训练过程/结果进行可视化（例如模型损失函数在训练过程中的变化趋势，或者参数的分布随训练批次的变化趋势等）。

## 2. 实现模型

### 2.1 数据集加载
可以通过飞桨自带的`paddle.vision.dataset`进行数据集加载，并且通过`paddle.vision.transforms`对数据进行预处理，例如对数据进行归一化。

In [ ]:
# !pip install optuna

In [1]:
import paddle
print(paddle.__version__)
import time 
import paddle.vision.transforms as T

transform = T.Compose([T.Normalize(mean=[127.5],
                                   std=[127.5],
                                   data_format='CHW')])

# 对数据进行归一化
train_dataset = paddle.vision.datasets.FashionMNIST(mode='train', transform=transform)
test_dataset = paddle.vision.datasets.FashionMNIST(mode='test', transform=transform)

2.1.3


### 2.2 模型搭建

使用paddle的接口搭建神经网络模型。以LeNet模型为例，该模型与1998年提出，包含了深度学习图像处理相关的基本模块。

LeNet模型一共有7层，包括2个**卷积层**，2个**池化层**和3个**全连接层**。通过连续使用卷积层和池化层提取图像特征。Paddle中提供了相应的接口，可以快速搭建网络模型：
1. 卷积层：`paddle.nn.Conv2D`；
2. 池化层：`paddle.nn.MaxPool2D`；
3. 全连接层：`paddle.nn.Linear`。

### **作业要求**：搭建神经网络，完成`__init__`和`forward`函数，并在README文件中详细描述所使用模型的信息。

In [2]:
import paddle.nn.functional as F
import paddle.nn as NN
class MyNet(paddle.nn.Layer):    
    def __init__(self):
        super(MyNet, self).__init__()
        self.layer1 = NN.Sequential(   
            NN.Conv2D(1, 16, kernel_size=5, padding=2),
            NN.BatchNorm2D(16), 
            NN.ReLU()) #16, 28, 28
        self.pool1=NN.MaxPool2D(2) #16, 14, 14
        self.layer2 = NN.Sequential(
            NN.Conv2D(16, 32, kernel_size=3),
            NN.BatchNorm2D(32),
            NN.ReLU())#32, 12, 12
        self.layer3 = NN.Sequential(
            NN.Conv2D(32, 64, kernel_size=3),
            NN.BatchNorm2D(64),
            NN.ReLU()) #64, 10, 10
        self.pool2=NN.MaxPool2D(2)  #64, 5, 5
        self.fc = NN.Linear(5*5*64, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        #print(out.shape)
        out=self.pool1(out)
        #print(out.shape)
        out = self.layer2(out)
        #print(out.shape)
        out=self.layer3(out)
        #print(out.shape)
        out=self.pool2(out)
        # print(out.shape)
        out = paddle.reshape(out,[-1,5*5*64])
        #print(out.shape)
        out = self.fc(out)
        return out

In [3]:
import paddle.nn.functional as F # 组网相关的函数，如conv2d, relu...
import numpy as np
from paddle.nn.layer.common import Dropout 
from paddle.vision.transforms import Compose, Resize, Transpose, Normalize, ToTensor
from paddle.vision.datasets import Cifar10

# 构建ResNet网络
# Sequential：顺序容器，子Layer将按构造函数参数的顺序添加到此容器中，传递给构造函数的参数可以Layers或可迭代的name Layer元组
from paddle.nn import Sequential, Conv2D, ReLU, MaxPool2D, Linear, Dropout, Flatten, BatchNorm2D, AvgPool2D

#构建模型
class Residual(paddle.nn.Layer):
    def __init__(self, in_channel, out_channel, use_conv1x1=False, stride=1):
        super().__init__()
        self.conv1 = Conv2D(in_channel, out_channel, kernel_size=3, padding=1, stride=stride)
        self.conv2 = Conv2D(out_channel, out_channel, kernel_size=3, padding=1)
        if use_conv1x1: #使用1x1卷积核
            self.conv3 = Conv2D(in_channel, out_channel, kernel_size=1, stride=stride)
        else:
            self.conv3 = None
        self.batchNorm1 = BatchNorm2D(out_channel)
        self.batchNorm2 = BatchNorm2D(out_channel)

    def forward(self, x):
        y = F.relu(self.batchNorm1(self.conv1(x)))
        y = self.batchNorm2(self.conv2(y))
        if self.conv3:
            x = self.conv3(x)
        out = F.relu(y+x) #核心代码
        return out
def ResNetBlock(in_channel, out_channel, num_layers, is_first=False):
    if is_first:
        assert in_channel == out_channel
    block_list = []
    for i in range(num_layers):
        if i == 0 and not is_first:
            block_list.append(Residual(in_channel, out_channel, use_conv1x1=True, stride=2))
        else:
            block_list.append(Residual(out_channel, out_channel))
    resNetBlock = Sequential(*block_list) #用*号可以把list列表展开为元素
    return resNetBlock

class ResNet50(paddle.nn.Layer):
    def __init__(self, num_classes=10):
        super().__init__()
        self.b1 = Sequential(
                    Conv2D(1, 64, kernel_size=7, stride=2, padding=3),
                    BatchNorm2D(64), 
                    ReLU(),
                    MaxPool2D(kernel_size=3, stride=2, padding=1))
        self.b2 = ResNetBlock(64, 64, 3, is_first=True)
        self.b3 = ResNetBlock(64, 128, 4)
        self.b4 = ResNetBlock(128, 256, 6)
        self.b5 = ResNetBlock(256, 512, 3)
        self.AvgPool = AvgPool2D(2)
        self.flatten = Flatten()
        self.Linear = Linear(512, num_classes)
        
    def forward(self, x):
        x = self.b1(x)
        x = self.b2(x)
        x = self.b3(x)
        x = self.b4(x)
        x = self.b5(x)
        x = self.AvgPool(x)
        x = self.flatten(x)
        x = self.Linear(x)
        return x
        
resnet = ResNet50(num_classes=10)
model = paddle.Model(resnet)
from paddle.static import InputSpec
input = InputSpec([None, 1, 28, 28], 'float32', 'image')
label = InputSpec([None, 1], 'int64', 'label')
model = paddle.Model(resnet, input, label)
model.summary()

---------------------------------------------------------------------------
 Layer (type)       Input Shape          Output Shape         Param #    
   Conv2D-1       [[1, 1, 28, 28]]     [1, 64, 14, 14]         3,200     
 BatchNorm2D-1   [[1, 64, 14, 14]]     [1, 64, 14, 14]          256      
    ReLU-1       [[1, 64, 14, 14]]     [1, 64, 14, 14]           0       
  MaxPool2D-1    [[1, 64, 14, 14]]      [1, 64, 7, 7]            0       
   Conv2D-2       [[1, 64, 7, 7]]       [1, 64, 7, 7]         36,928     
 BatchNorm2D-2    [[1, 64, 7, 7]]       [1, 64, 7, 7]           256      
   Conv2D-3       [[1, 64, 7, 7]]       [1, 64, 7, 7]         36,928     
 BatchNorm2D-3    [[1, 64, 7, 7]]       [1, 64, 7, 7]           256      
  Residual-1      [[1, 64, 7, 7]]       [1, 64, 7, 7]            0       
   Conv2D-4       [[1, 64, 7, 7]]       [1, 64, 7, 7]         36,928     
 BatchNorm2D-4    [[1, 64, 7, 7]]       [1, 64, 7, 7]           256      
   Conv2D-5       [[1, 64, 7, 7]]   

{'total_params': 21305482, 'trainable_params': 21275018}

## 3. 模型配置与训练
可以使用`Model`搭建实例，然后使用`model.prepare`接口进行模型的配置，比如优化器、损失函数和评价指标等，也可以使用其他方法配置和训练模型。

In [4]:
BATCH_SIZE = 64
train_loader = paddle.io.DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE)
test_loader = paddle.io.DataLoader(test_dataset, batch_size=BATCH_SIZE)
# 为模型训练做准备，设置优化器，损失函数和精度计算方式
learning_rate = 0.001
loss_fn = paddle.nn.CrossEntropyLoss()
opt = paddle.optimizer.Adam(learning_rate=learning_rate, parameters=model.parameters())
model.prepare(optimizer=opt, loss=loss_fn, metrics=paddle.metric.Accuracy())

log_dir = './log/resnett50'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
callback_train = paddle.callbacks.VisualDL(log_dir=log_dir)

# 启动模型训练，指定训练数据集，设置训练轮次，设置每次数据集计算的批次大小，设置日志格式
model.fit(train_loader, test_loader, batch_size=64, epochs=20, eval_freq= 5, verbose=1, callbacks=callback_train)
model.evaluate(test_loader, verbose=1)


The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/20


/Users/vincent/.pyenv/versions/3.6.3/envs/torch_3.6.3/lib/python3.6/site-packages/paddle/nn/layer/norm.py:641: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")


step 938/938 [==============================] - loss: 0.3347 - acc: 0.8354 - 2s/step           
Eval begin...
step 157/157 [==============================] - loss: 0.3244 - acc: 0.8696 - 835ms/step          
Eval samples: 10000
Epoch 2/20
step 540/938 [================>.............] - loss: 0.2648 - acc: 0.8790 - ETA: 15:07 - 2s/step

In [ ]:
model.save('resnet-50')  # save for inference

In [ ]:
# model = paddle.Model(MyNet()) # 用Model封装模型
# from paddle.vision.models import LeNet
# model = paddle.Model(LeNet())
# lr = 0.001
# optimizer = paddle.optimizer.Adam(learning_rate=lr, parameters=model.parameters())
# optimizer = paddle.optimizer.Momentum(learning_rate=lr, parameters=model.parameters(), momentum=0.9)
# optimizer = paddle.optimizer.SGD(learning_rate=lr, parameters = model.parameters())
# 配置模型
# model.prepare(paddle.optimizer.Adam(learning_rate=0.001, parameters=model.parameters()),
#               paddle.nn.CrossEntropyLoss(),
#               paddle.metric.Accuracy())

In [ ]:
# find the best hyperparams
# import optuna
# from optuna import trial
# def objective(trial):
    # optimizer = None
    # lr = trail.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    # optimizer_name = trail.suggest_categorical("optimizer", ["optim_Mom", "optim_SGD", 'optim_Adam','optim_Adagrad','optim_RMS'])
    # if optimizer_name == 'optim_Mom':
    #     optimizer = paddle.optimizer.Momentum(learning_rate=lr, parameters=model.parameters(), momentum=0.9)
    # if optimizer_name == 'optim_SGD':
    #     optimizer = paddle.optimizer.SGD(learning_rate=lr, parameters = model.parameters())
    # if optimizer_name == 'optim_Adam':
    #     optimizer = paddle.optimizer.Adam(learning_rate=lr, parameters=model.parameters())
    # if optimizer_name == 'optim_Adagrad':
    #     optimizer = paddle.optimizer.Adagrad(learning_rate=lr, parameters=model.parameters())
    # if optimizer_name == 'optim_RMS':
    #     optimizer = paddle.optimizer.RMSProp(learning_rate=lr, parameters=model.parameters(), weight_decay=0.01)
       # Generate the optimizers.
#     optim = paddle.optimizer
#     optimizer_name = trial.suggest_categorical("optimizer", ['Momentum', "Adam", "RMSprop", "SGD", 'Adagrad'])
#     lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
#     optimizer = getattr(optim, optimizer_name)(parameters=model.parameters(), learning_rate=lr)
# study = optuna.create_study()
# study.optimize(objective, n_trials=10)
# best_params = study.best_params
# print(best_params)

### **作业要求**：完成训练过程可视化的相关函数，训练模型并保存可视化结果。

# 作业模型运行优化日志：
- 第一次 cnn train，learning_rate=0.001, Adam evaluation：0.9 左右，然后再怎么train也不动了
- 第二次 使用默认的LeNet，lr = 0.001, Adam evaluation: {'loss': [0.7828319], 'acc': 0.8948}
- 第三次 使用lenet Momentum lr=0.001, evaluation: {'loss': [0.1281017], 'acc': 0.8981}
- 第四次 使用mynet Momentum lr=0.001, {'loss': [0.39709407], 'acc': 0.9044}
- 第五次 使用mynet SGD lr=0.001：{'loss': [0.2486805], 'acc': 0.892}
- 第六次 使用mynet SGD lr=0.01： {'loss': [4.3855166], 'acc': 0.8912}
- 第七次 使用mynet Momentum lr=0.01：{'loss': [0.011295634], 'acc': 0.9005}
- 尝试使用 optuna 来寻找最优参数, 最终发现不能够使用啊，paddle 支持

In [ ]:
import matplotlib.pyplot as plt
import os
# 参数的分布图， 损失函数， acc
def DrawLossFunction():
    pass
log_dir = './log/resnet18'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
callback_train = paddle.callbacks.VisualDL(log_dir=log_dir)

model.fit(train_dataset, test_dataset, epochs=50, batch_size=64, verbose=1, callbacks=callback_train)

In [ ]:
model.evaluate(test_dataset, verbose=1) # batch_size=64,